In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1. 加载KMNIST数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.KMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.KMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. 构建神经网络模型
class KMNISTModel(nn.Module):
    def __init__(self):
        super(KMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = KMNISTModel()

# 3. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 训练模型
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)

# 5. 调整模型结构
# 例如，增加隐藏层或改变神经元数量
class EnhancedKMNISTModel(nn.Module):
    def __init__(self):
        super(EnhancedKMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 128 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

enhanced_model = EnhancedKMNISTModel()
optimizer = optim.Adam(enhanced_model.parameters(), lr=0.001)
train_model(enhanced_model, train_loader, criterion, optimizer)

# 6. 调试超参数
# 例如，尝试不同的学习率和批次大小
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

# 测试不同的学习率
learning_rates = [0.001, 0.0001, 0.00001]
for lr in learning_rates:
    print(f"Testing with learning rate: {lr}")
    model = KMNISTModel()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_model(model, train_loader, criterion, optimizer)
    evaluate_model(model, test_loader)

# 测试不同的批次大小
batch_sizes = [32, 64, 128]
for batch_size in batch_sizes:
    print(f"Testing with batch size: {batch_size}")
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    model = KMNISTModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, train_loader, criterion, optimizer)
    evaluate_model(model, test_loader)

100%|██████████| 18.2M/18.2M [00:04<00:00, 3.64MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 341kB/s]
100%|██████████| 3.04M/3.04M [00:02<00:00, 1.03MB/s]
100%|██████████| 5.12k/5.12k [00:00<00:00, 2.63MB/s]


Epoch 1/5, Loss: 0.2662619527524659
Epoch 2/5, Loss: 0.07432799635262394
Epoch 3/5, Loss: 0.042554423830701286
Epoch 4/5, Loss: 0.025192607891187865
Epoch 5/5, Loss: 0.018207954965852466
Epoch 1/5, Loss: 0.22007539753776306
Epoch 2/5, Loss: 0.05481431783419718
Epoch 3/5, Loss: 0.029810027338687015
Epoch 4/5, Loss: 0.020489288522555625
Epoch 5/5, Loss: 0.016408143914606706
Testing with learning rate: 0.001
Epoch 1/5, Loss: 0.27525460927773004
Epoch 2/5, Loss: 0.07273213460898476
Epoch 3/5, Loss: 0.041259672856199116
Epoch 4/5, Loss: 0.02528292230095343
Epoch 5/5, Loss: 0.017220813704966774
Accuracy: 94.3%
Testing with learning rate: 0.0001
Epoch 1/5, Loss: 0.7191057412990375
Epoch 2/5, Loss: 0.2902733348389424
Epoch 3/5, Loss: 0.19739649430123854
Epoch 4/5, Loss: 0.15382122050430666
Epoch 5/5, Loss: 0.12446877976923164
Accuracy: 90.25%
Testing with learning rate: 1e-05
Epoch 1/5, Loss: 1.729863862175423
Epoch 2/5, Loss: 0.8831345234344254
Epoch 3/5, Loss: 0.6771909989146535
Epoch 4/5, L